In [1]:
#THIS NOTEBOOK WILL COPY OVER NER ROW LAYERS
# PRE-STEP - DOWNLOAD NER ROW LAYER FROM ONLINE AND ADD DATE OF DOWNLOAD, THEN ADD TO NER ROW Layers gdb

In [2]:
#TASK 1 - SET YOUR SOURCE AND TARGET DATA SETS

In [3]:
source_feature = r'C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb\NER_ROW_ply_07022024'

target_feature = r'C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb\DBO_EGDB_UseLocations_py'

target_feature_workspace = r'C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb'

county_feature = r'C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb\USA_Counties'

NER_CNTY_JN = r'C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb/NER_CNTY_Join4'

source_feature_workspace = r'C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb'

In [4]:
#TASK 2 - EXAMINE TEMPLATE VS NER ROW LAYER AND ADD AY TEMPLATE FIELDS TO THE NER ROW LAYER

In [5]:
import arcpy

# List fields in the source and target feature classes
source_fields = arcpy.ListFields(source_feature)
target_fields = arcpy.ListFields(target_feature)

# Create a set of field names in the source feature class
source_field_names = set(field.name for field in source_fields)

# Loop through the target fields and add any missing fields to the source feature class
for field in target_fields:
    if field.name not in source_field_names:
        try:
            # Add the field to the source feature class
            arcpy.AddField_management(
                in_table=source_feature,
                field_name=field.name,
                field_type=field.type,
                field_precision=field.precision,
                field_scale=field.scale,
                field_length=field.length,
                field_alias=field.aliasName,
                field_is_nullable=field.isNullable,
                field_is_required=field.required,
                field_domain=field.domain
            )
            print(f"Field {field.name} added to the source feature class.")
        except arcpy.ExecuteError as e:
            print(f"Error adding field {field.name}: {e}")
    else:
        print(f"Field {field.name} already exists in the source feature class.")

print("Field comparison and addition completed.")


Field OBJECTID already exists in the source feature class.
Field Shape already exists in the source feature class.
Field FEATUREID already exists in the source feature class.
Field USE_BNDY_NO already exists in the source feature class.
Field REC_STATE already exists in the source feature class.
Field REC_CNTY already exists in the source feature class.
Field BK_VOL already exists in the source feature class.
Field PAGE already exists in the source feature class.
Field RECEPT_NO already exists in the source feature class.
Field REC_DATE already exists in the source feature class.
Field INS_DATE already exists in the source feature class.
Field SOURCEDATE already exists in the source feature class.
Field SURV_DATE already exists in the source feature class.
Field PLAT_TITLE already exists in the source feature class.
Field GRANTOR already exists in the source feature class.
Field GRANTEE already exists in the source feature class.
Field LEGAL_ACRE already exists in the source feature cl

In [6]:
#TASK 3 - JOIN YOUR LAYER TO THE COUNTY DATA IN ORDER TO CALCULATE YOUR CNTY AND STATE VALUES

In [7]:
import arcpy

# Define the paths to the feature classes
output_feature = NER_CNTY_JN

# Perform the spatial join
arcpy.analysis.SpatialJoin(
    target_features=source_feature, 
    join_features=county_feature, 
    out_feature_class=output_feature,
    join_type="KEEP_COMMON"
)

# Calculate fields REC_CNTY and REC_STATE
arcpy.management.CalculateField(output_feature, "REC_CNTY", "!NAME!", "PYTHON3")
arcpy.management.CalculateField(output_feature, "REC_STATE", "!STATE_ABBR!", "PYTHON3")

print("Spatial join and field calculation completed successfully.")


Spatial join and field calculation completed successfully.


In [8]:
#TASK 4 - EXAMINE TEMPLATE DOMAINS AND THE FIELDS THEY APPLY TO 

In [9]:
import arcpy

# Get the workspace (geodatabase) from the target feature
workspace = target_feature_workspace

# Set the workspace
arcpy.env.workspace = workspace

# List all fields in the target feature
fields = arcpy.ListFields(target_feature)

# Get all domains in the workspace
domains = arcpy.da.ListDomains(workspace)

# Create a dictionary to map domain names to their fields
domain_field_map = {}

# Iterate through fields and populate the domain-field map
for field in fields:
    if field.domain:
        if field.domain not in domain_field_map:
            domain_field_map[field.domain] = []
        domain_field_map[field.domain].append(field.name)

# Function to print domain values
def print_domain_values(domain):
    if domain:
        print(f"Domain Name: {domain.name}")
        if domain.domainType == 'CodedValue':
            coded_values = domain.codedValues
            for code, description in coded_values.items():
                print(f" - Code: {code}")
        elif domain.domainType == 'Range':
            print(f" - Range: {domain.range[0]} - {domain.range[1]}")
    else:
        print("No domain assigned.")

# Iterate through domains and print their associated fields and values
for domain_name, field_names in domain_field_map.items():
    domain_obj = next((d for d in domains if d.name == domain_name), None)
    print(f"Domain: {domain_name}")
    print("Used in fields: " + ", ".join(field_names))
    print_domain_values(domain_obj)
    print()

print("Domain listing completed.")


Domain listing completed.


In [10]:
#TASK 5 - POPULATE MAPMETHOD BASED ON A SET OF RULES

In [11]:
#TASK 5A - Examine common words used in your DATA_SOURCE field to determine appropriate categories to use when calculating MAPMETHOD

In [12]:
import arcpy
from collections import Counter
import re

# Define the path to the NER_CNTY_JN feature class
ner_cnty_jn = NER_CNTY_JN

# Initialize a Counter to count word frequencies
word_counter = Counter()

# Define a regular expression pattern to match words
word_pattern = re.compile(r'\b\w+\b')

# Initialize a counter for the word "survey"
survey_count = 0

# Create a search cursor to read the DATA_SOURCE field
with arcpy.da.SearchCursor(ner_cnty_jn, ["DATA_SOURCE"]) as cursor:
    for row in cursor:
        data_source = row[0]
        if data_source:
            # Find all words in the DATA_SOURCE field and update the counter
            words = word_pattern.findall(data_source.lower())
            word_counter.update(words)
            
            # Check for the word "survey"
            if "survey" in words:
                survey_count += 1

# Get the most common words and their counts
most_common_words = word_counter.most_common(20)  # Change 10 to the desired number of top words

# Print the most common words
print("Most common words in the DATA_SOURCE field:")
for word, count in most_common_words:
    print(f"{word}: {count}")

# Print the count of the word "survey"
print(f"\nThe word 'survey' was found {survey_count} times in the DATA_SOURCE field.")

print("Summary of word frequencies completed.")


Most common words in the DATA_SOURCE field:
and: 196
data: 178
park: 96
description: 79
on: 77
centerlines: 72
map: 69
legal: 68
aerial: 66
paper: 65
roads: 63
gis: 60
state: 56
based: 55
of: 55
mainedot: 54
public: 54
to: 39
with: 38
plat: 37

The word 'survey' was found 13 times in the DATA_SOURCE field.
Summary of word frequencies completed.


In [13]:
#TASK 5B - CALCULATE MAPMETHOD BASED ON TEXT IN OLD DATA_SOURCE FIEDL

In [14]:
import arcpy
from collections import Counter

# Define the path to the source feature class
source_feature = NER_CNTY_JN

# Add the MAPMETHOD field if it doesn't exist
if not arcpy.ListFields(source_feature, "MAPMETHOD"):
    arcpy.management.AddField(source_feature, "MAPMETHOD", "TEXT")

# Define an update cursor to update the MAPMETHOD field based on DATA_SOURCE
with arcpy.da.UpdateCursor(source_feature, ["DATA_SOURCE", "MAPMETHOD"]) as cursor:
    for row in cursor:
        data_source = row[0]
        if data_source is not None:
            data_source_lower = data_source.lower()
            if "survey" in data_source_lower or "plat" in data_source_lower:
                row[1] = "Surveyed/GPS/Geodetically Derived"
            elif "digitiz" in data_source_lower or "aerial" in data_source_lower or "paper" in data_source_lower:
                row[1] = "Digitized"
            elif "cogo" in data_source_lower or "legal" in data_source_lower:
                row[1] = "COGO"
            else:
                row[1] = "Other"
        else:
            row[1] = "Unknown"
        cursor.updateRow(row)

print("Field calculation for MAPMETHOD completed.")

# Count the occurrences of each value in the MAPMETHOD field
mapmethod_counter = Counter()

# Use a search cursor to count the MAPMETHOD values
with arcpy.da.SearchCursor(source_feature, ["MAPMETHOD"]) as cursor:
    for row in cursor:
        mapmethod_counter[row[0]] += 1

# Print the summary of MAPMETHOD values
print("\nSummary of MAPMETHOD field values:")
for method, count in mapmethod_counter.items():
    print(f"{method}: {count}")

print("Summary of MAPMETHOD values completed.")



Field calculation for MAPMETHOD completed.

Summary of MAPMETHOD field values:
Surveyed/GPS/Geodetically Derived: 71
Unknown: 13
Digitized: 96
COGO: 36
Other: 226
Summary of MAPMETHOD values completed.


In [15]:
#TASK 6 - HANDLE MAPSOURCE FIELD USING DATA_SOURCE TEXT 

In [16]:
import arcpy
from collections import Counter

# Define the path to the source feature class
source_feature = NER_CNTY_JN

# Add the MAPSOURCE field if it doesn't exist
if not arcpy.ListFields(source_feature, "MAPSOURCE"):
    arcpy.management.AddField(source_feature, "MAPSOURCE", "TEXT")

# Define an update cursor to update the MAPSOURCE field based on DATA_SOURCE
with arcpy.da.UpdateCursor(source_feature, ["DATA_SOURCE", "MAPSOURCE"]) as cursor:
    for row in cursor:
        data_source = row[0]
        if data_source is not None:
            data_source_lower = data_source.lower()
            if "plat" in data_source_lower or "legal" in data_source_lower:
                row[1] = "Document - Survey Plat, Unrecorded"
            elif "paper" in data_source_lower:
                row[1] = "Digital - Georeferenced Map or Plat"
            elif "aerial" in data_source_lower:
                row[1] = "Digital - Aerial Photo"
            elif "data" in data_source_lower or "cad" in data_source_lower:
                row[1] = "Digital - CAD File"
            elif "gis" in data_source_lower:
                row[1] = "Digital - Other"
            else:
                row[1] = "Document - Other"
        else:
            row[1] = "Not Identified"
        cursor.updateRow(row)

print("Field calculation for MAPSOURCE completed.")

# Count the occurrences of each value in the MAPMETHOD field
mapmethod_counter = Counter()

# Use a search cursor to count the MAPSOURCE values
with arcpy.da.SearchCursor(source_feature, ["MAPSOURCE"]) as cursor:
    for row in cursor:
        mapmethod_counter[row[0]] += 1

# Print the summary of MAPSOURCE values
print("\nSummary of MAPSOURCE field values:")
for method, count in mapmethod_counter.items():
    print(f"{method}: {count}")

print("Summary of MAPSOURCE values completed.")



Field calculation for MAPSOURCE completed.

Summary of MAPSOURCE field values:
Document - Survey Plat, Unrecorded: 111
Not Identified: 13
Digital - Georeferenced Map or Plat: 35
Digital - CAD File: 162
Document - Other: 52
Digital - Aerial Photo: 48
Digital - Other: 21
Summary of MAPSOURCE values completed.


In [17]:
#TASK 7 - HANDLE ESTACCURACY USING VALUES IN DataQuality field

In [18]:
import arcpy
from collections import Counter

# Define the path to the source feature class
source_feature = NER_CNTY_JN

# Add the ESTACCURACY field if it doesn't exist
if not arcpy.ListFields(source_feature, "ESTACCURACY"):
    arcpy.management.AddField(source_feature, "ESTACCURACY", "TEXT")

# Define an update cursor to update the ESTACCURACY field based on DataQuality
with arcpy.da.UpdateCursor(source_feature, ["DataQuality", "ESTACCURACY"]) as cursor:
    for row in cursor:
        data_quality = row[0]
        if data_quality is not None:
            if data_quality.lower() == "low":
                row[1] = "Low >10m"
            elif data_quality.lower() == "medium":
                row[1] = "Medium +/-3m-10m"
            elif data_quality.lower() == "high":
                row[1] = "High +/-<3m"
        else:
            row[1] = "Low >10m"  # Set null values to "Low >10m"
        cursor.updateRow(row)

print("Field calculation for ESTACCURACY completed.")

# Count the occurrences of each value in the ESTACCURACY field
mapmethod_counter = Counter()

# Use a search cursor to count the ESTACCURACY values
with arcpy.da.SearchCursor(source_feature, ["ESTACCURACY"]) as cursor:
    for row in cursor:
        mapmethod_counter[row[0]] += 1

# Print the summary of ESTACCURACY values
print("\nSummary of ESTACCURACY field values:")
for method, count in mapmethod_counter.items():
    print(f"{method}: {count}")

print("Summary of ESTACCURACY values completed.")

Field calculation for ESTACCURACY completed.

Summary of ESTACCURACY field values:
Low >10m: 310
High +/-<3m: 45
Medium +/-3m-10m: 87
Summary of ESTACCURACY values completed.


In [19]:
#TASK 8 - HANDLE DATAACCESS AND PUBLICDISPLAY BY SETTING VALUES - THESE ARE THE SAME ACROSS ALL

In [20]:
import arcpy
from collections import Counter

# Define the path to the source feature class
source_feature = NER_CNTY_JN

# Define an update cursor to update the fields based on null values
with arcpy.da.UpdateCursor(source_feature, ["DATAACCESS", "PUBLICDISPLAY"]) as cursor:
    for row in cursor:
        if row[0] is None:
            row[0] = "Internal NPS Only"  # Set DATAACCESS to "Internal NPS Only" if null
        if row[1] is None:
            row[1] = "No Public Map Display"  # Set PUBLICDISPLAY to "No Public Map Display" if null
        cursor.updateRow(row)

print("Field updates completed for DATAACCESS and PUBLICDISPLAY.")


Field updates completed for DATAACCESS and PUBLICDISPLAY.


In [21]:
#TASK 9 - HANDLE A SET OF FIELDS WITH A ONE TO ONE RELATIONSHIP
    # PATTERN IS AS FOLLOWS
        #NEW FIELD: OLD FIELD
    #ADD ANY ADDITIONAL ONE TO ONES AS NEEDED INTO THE LIST

In [32]:
import arcpy

# Define the path to the source feature class
source_feature1 = r'C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb\NER_CNTY_Join4'
arcpy.env.workspace = source_feature_workspace

def calculate_fields(feature_class):
    try:
        # List of fields to calculate
        fields_to_calculate = [
            ("FEATUREID", "!Location_Key!"),
            ("TRACT_ID", "str(!Tract!)[0:20]"),  # Adjusted to use string slicing for text field
            ("USE_BNDY_NO", "str(!ROW_ID!)[0:20]"),  # Adjusted to use string slicing for text field
            ("UNITCODE", "!PARK_ALPHA!"),
            ("GRANTEE", "!PERMITTEE!")
        ]

        # Get the list of field names in the feature class
        existing_fields = [field.name for field in arcpy.ListFields(feature_class)]

        # Start an edit session and apply field calculations
        with arcpy.da.Editor(arcpy.env.workspace) as edit:
            for output_field, expression in fields_to_calculate:
                if output_field in existing_fields:
                    arcpy.CalculateField_management(
                        in_table=feature_class,
                        field=output_field,
                        expression=expression,
                        expression_type='PYTHON3'
                    )
                else:
                    print(f"Field {output_field} not found in {feature_class}. Skipping calculation.")

        print(f"Calculation of fields for {feature_class} completed successfully.")

    except arcpy.ExecuteError:
        print(f"Error executing arcpy operation: {arcpy.GetMessages(2)}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")

# Call the function with your feature class
if __name__ == "__main__":
    calculate_fields(source_feature1)


Calculation of fields for C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb\NER_CNTY_Join4 completed successfully.


In [23]:
#TASK 10 - CONCATENATE EXTRANEOUS TEXT AND NOTES FIELDS INTO NOTES1 FIELD

In [24]:
#pre task - in order to handle null values in concatenate expression, set the fields equal to "null"
# Pre-cell to handle NULL values in specified fields

import arcpy

# List of fields to check and replace NULL values
fields_to_check = [
    "ExtraMapNotes",
    "ROW_short_description",
    "Notes"
]

# Iterate over each field and replace None with "null"
for field in fields_to_check:
    try:
        # Update cursor to find and replace NULL values
        with arcpy.da.UpdateCursor(source_feature, [field]) as cursor:
            for row in cursor:
                if row[0] is None:
                    row[0] = "null"
                    cursor.updateRow(row)
        
        print(f"Replaced NULL values with 'null' in field: {field}")

    except arcpy.ExecuteError:
        print(f"Error executing arcpy operation for field {field}: {arcpy.GetMessages(2)}")
    except Exception as e:
        print(f"Unexpected error occurred for field {field}: {e}")


Replaced NULL values with 'null' in field: ExtraMapNotes
Replaced NULL values with 'null' in field: ROW_short_description
Replaced NULL values with 'null' in field: Notes


In [25]:
import arcpy

# Define the path to the source feature class
source_feature = NER_CNTY_JN
arcpy.env.workspace = source_feature_workspace

def calculate_notes1_concatenation(feature_class):
    try:
        # Calculate NOTES1 field using concatenation of specified fields
        arcpy.CalculateField_management(
            in_table=feature_class,
            field="NOTES1",
            expression='(!ExtraMapNotes! + ", " if !ExtraMapNotes! else "") + (!ROW_short_description! + ", " if !ROW_short_description! else "") + (!Notes! if !Notes! else "")[:254]',
            expression_type='PYTHON3'
        )

        print(f"Concatenation for NOTES1 field in {feature_class} completed successfully.")

    except arcpy.ExecuteError:
        print(f"Error executing arcpy operation: {arcpy.GetMessages(2)}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")

# Call the function with your feature class
if __name__ == "__main__":
    calculate_notes1_concatenation(source_feature)


Concatenation for NOTES1 field in C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb/NER_CNTY_Join4 completed successfully.


In [26]:
#TASK 11 - Using a blank template gdb pathway, list out any extraneous fields added to your source_feature

In [27]:
import arcpy

def compare_fields(source_fc, target_fc):
    try:
        # Get the list of fields in the source feature class
        source_fields = [field.name for field in arcpy.ListFields(source_fc)]

        # Get the list of fields in the target feature class
        target_fields = [field.name for field in arcpy.ListFields(target_fc)]

        # Find fields in source_fc that are not in target_fc
        fields_not_in_target = set(source_fields) - set(target_fields)

        # Convert set to list for printing
        fields_not_in_target_list = list(fields_not_in_target)

        if fields_not_in_target_list:
            print(f"Fields in {source_fc} not present in {target_fc}:")
            for field in fields_not_in_target_list:
                print(field)
        else:
            print(f"All fields in {source_fc} are also present in {target_fc}.")

        return fields_not_in_target_list

    except arcpy.ExecuteError:
        print(f"Error executing arcpy operation: {arcpy.GetMessages(2)}")
        return None
    except Exception as e:
        print(f"Unexpected error occurred: {e}")
        return None

# Call the function with your source and target feature classes
if __name__ == "__main__":
    fields_not_in_target = compare_fields(source_feature, target_feature)
    if fields_not_in_target is not None:
        # Now fields_not_in_target contains the list of fields not in target_feature
        print("List of fields not present in target_feature:")
        print(fields_not_in_target)


Fields in C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb/NER_CNTY_Join4 not present in C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb\DBO_EGDB_UseLocations_py:
STATE_ABBR
AppEmail
NAME
ProjectName
Fee
AppCity
Parcel
AppZip
MilePost
Date_Complete_Application
App_Phone
FIPS
DATA_SOURCE
Expiration_Date
Join_Count
Access
TARGET_FID
Active
Shape_Length_1
POP20_SQMI
Effective_Date
DataQuality
ROW_Authority
Location_Key
ParkAuthority
LocationState
Area_sqft
DescriptionofProperty
COUNTY_FIPS
ROW_ParkContact
PARK_ALPHA
PERMITTEE
NCR_AdminFilePathway
ParkUnit_full
ROW_Width
Instrument
Ownership
LonTextDescription
POPULATION
Rights
POPULATION_2020
ROW_adverb
Acres
ParkContacEmail
Encumbrance_Subtype
CostRecovery
PEPC
ROW_short_description
ExtraMapNotes
App_Address
STATE_NAME
LongDescription
SQMI
POP_SQMI
Tract
Shape_Area_1
AppState
STATE_FIPS
ROW_ID
Applicant
Notes
List of fields not present in target_feature:
['STATE_ABBR'

In [28]:
#TASK 12 - create a final copy for the upload to the online versioning system; and then remove extraneous fields

In [33]:
import arcpy

try:
    # Create a copy of source_feature in source_feature_workspace
    copy_fc_path = arcpy.FeatureClassToFeatureClass_conversion(NER_CNTY_JN, source_feature_workspace, "NER_ROW_forpaste07032024").getOutput(0)
    print(f"Copy of {NER_CNTY_JN} created as 'NER_ROW_forpaste07032024' in {source_feature_workspace}.")

    # Remove fields_not_in_target from the copied feature class
    arcpy.DeleteField_management(copy_fc_path, fields_not_in_target)
    print(f"Removed specified fields from {copy_fc_path}.")

except arcpy.ExecuteError:
    print(f"Error executing arcpy operation: {arcpy.GetMessages(2)}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")


Copy of C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb/NER_CNTY_Join4 created as 'NER_ROW_forpaste07032024' in C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb.
Removed specified fields from C:\Users\hdean\Desktop\GDB_Conversion_2024\GDB_Conversion6252024\NER_ROW_Layers.gdb\NER_ROW_forpaste07032024.
